<a href="https://colab.research.google.com/github/edgarcancinoe/sapienza_cv/blob/main/vision_tutorial_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

## ⚠️ Important: Read Before Running  

This notebook is set to **execution-only mode**, meaning you can **run all the cells**, but you **cannot modify** the code directly.  

If you want to **edit** or **test your own modifications**, follow these steps:  

1️⃣ **Go to**: `File` → `Save a copy in Drive`  
2️⃣ This will create a **personal copy** of the notebook in your own Google Drive.  
3️⃣ You can now **edit and experiment freely** without affecting the original notebook.  

This ensures that everyone has access to a clean and structured version of the notebook while allowing individual experimentation 🚀

In [1]:
import cv2
import ipywidgets as widgets
import matplotlib.image as mpim
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import random
import requests
import urllib.request

from google.colab.patches import cv2_imshow
from io import BytesIO
from IPython.display import display, clear_output
from ipywidgets import interact, IntSlider, FloatSlider, FloatRangeSlider, Dropdown
from matplotlib.gridspec import GridSpec
from matplotlib.patches import FancyArrowPatch
from PIL import Image
from prettytable import PrettyTable
from scipy import signal
from scipy.fft import fft, fftfreq, fftshift
from skimage import io, color

# Image loading

We load an image from the web and use it as test image for our experiments

In [ ]:
!wget -O image.jpg "https://upload.wikimedia.org/wikipedia/commons/thumb/d/de/Colosseo_2020.jpg/1200px-Colosseo_2020.jpg"

Visualize the image with OpenCV

In [ ]:
image = cv2.imread("image.jpg")
cv2_imshow(image)

Visualize the image through Matplotlib

In [ ]:
plt.imshow(image)
plt.axis("off")
plt.show()

Why do we see the image with altered colors?

Because OpenCV stores the images in the BGR format.

We need to treat it in RGB format with Matplotlib

In [ ]:
b,g,r = cv2.split(image)
rgb_image = cv2.merge([r,g,b])

Revisualize it!

In [ ]:
plt.imshow(rgb_image)
plt.axis("off")
plt.show()

image = rgb_image

# Image filtering

Filtering: Form a new image whose pixel values are a combination of
the original pixel values.


It is used to:
- Remove noise
- Detect edges
- Blur the image


## Mean filter

The Mean Filter is a simple smoothing filter that replaces each pixel’s value
with the average of its neighboring pixels by usign a kernel. The higher the kernel size, the higher the blur in the resulting image.

$$h[x,y] = \frac{1}{N}\sum_{i,j}f[x+i,y+j]$$

where:
- $i$ is the original image
- $h[x,y]$ is the output pixel value at location $(x,y)$
- $N$ is the total number of pixels in the kernel

The kernel must be odd to have a well-defined central pixel in the kernel, making calculations symmetrical.

Properties:
- blurring ✅
- no edge preservation ❌

In [ ]:
mean_kernel_slider = widgets.IntSlider(value=5, min=3, max=21, step=2, description="Kernel Size")

def mean_filter(kernel_size):
    # apply Mean Filter
    mean_filtered = cv2.blur(image, (kernel_size, kernel_size))

    plt.figure(figsize=(18,8))

    plt.subplot(1, 2, 1)
    plt.imshow(image)
    plt.title("Original Image")
    plt.axis("off")

    plt.subplot(1, 2, 2)
    plt.imshow(mean_filtered)
    plt.title(f"Mean Filter (Kernel={kernel_size}x{kernel_size})")
    plt.axis("off")

    plt.show()

interactive_mean = widgets.interactive(mean_filter, kernel_size=mean_kernel_slider)
display(interactive_mean)

## Gaussian filter

The Gaussian Filter is similar to the Mean Filter, but instead of averaging all pixels equally, it applies a weighted average where nearby pixels have higher importance.

$$h[m,n] = \sum_{k,l}g[k,l]f[m+k,n+l]$$

The kernel must be odd to have a well-defined central pixel in the kernel, making calculations symmetrical

where:
- $h$ is the output image
- $g$ is the spatial weghting to favro nearby pixels
- $f$ is the original image

Properties:
- smoother blurring ✅
- small edge preservation 😐

In [ ]:
gaussian_kernel_slider = widgets.IntSlider(value=5, min=3, max=21, step=2, description="Kernel Size")

def gaussian_filter(kernel_size):
    # apply Gaussian Filter
    gaussian_filtered = cv2.GaussianBlur(image, (kernel_size, kernel_size), 0)

    plt.figure(figsize=(18,8))

    plt.subplot(1, 2, 1)
    plt.imshow(image)
    plt.title("Original Image")
    plt.axis("off")

    plt.subplot(1, 2, 2)
    plt.imshow(gaussian_filtered)
    plt.title(f"Gaussian Filter (Kernel={kernel_size}x{kernel_size})")
    plt.axis("off")

    plt.show()

interactive_gaussian = widgets.interactive(gaussian_filter, kernel_size=gaussian_kernel_slider)
display(interactive_gaussian)

## Bilateral filter

The Bilateral Filter is a non-linear, edge-preserving, and noise-reducing filter. Unlike Mean and Gaussian filters, which blur edges, the Bilateral Filter smooths noise while keeping edges sharp. Nearby pixels contribute more (like Gaussian filtering). Pixels with similar intensity contribute more (preserving edges).

**Remember**: edges are characterized by rapid intensity change in the image

$$h[m,n] = \frac{1}{W_{mn}}\sum_{k,l}g[k,l]r_{mn}[k,l]f[m+k,n+l]$$

where:
- $h$ is the output image
- $g$ is the spatial weighting to favor *nearby* pixels
- $r$ is the intensity range weighting to favor *similar* pixels
- $\frac{1}{W_{mn}}$ is a normalization factor
- $f$ is the original image

Properties:
- smooth blurring ✅
- edge preservation ✅

bilateralFilter function Parameters:

- Diameter: Size of the pixel neighborhood used for filtering. A larger d means more neighboring pixels influence the smoothing.

- Sigma Color: How much intensity differences matter. Higher values mean stronger smoothing, even for pixels with large intensity differences.


- Sigma Space: How much spatial distance matters. Higher values allow more distant pixels to influence the smoothing.


In [ ]:
diameter_slider = widgets.IntSlider(value=9, min=1, max=20, step=2, description="Diameter")
sigma_color_slider = widgets.IntSlider(value=75, min=10, max=200, step=5, description="Sigma Color")
sigma_space_slider = widgets.IntSlider(value=75, min=10, max=200, step=5, description="Sigma Space")

def bilateral_filter(diameter, sigma_color, sigma_space):
  # apply Bilateral Filter
    bilateral_filtered = cv2.bilateralFilter(image, diameter, sigma_color, sigma_space)

    plt.figure(figsize=(18,8))

    plt.subplot(1, 2, 1)
    plt.imshow(image)
    plt.title("Original Image")
    plt.axis("off")

    plt.subplot(1, 2, 2)
    plt.imshow(bilateral_filtered)
    plt.title(f"Bilateral Filter (d={diameter}, σc={sigma_color}, σs={sigma_space})")
    plt.axis("off")

    plt.show()

interactive_plot = widgets.interactive(bilateral_filter,
                                       diameter=diameter_slider,
                                       sigma_color=sigma_color_slider,
                                       sigma_space=sigma_space_slider)
display(interactive_plot)

## Convolution (nothing else than what done so far)

Convolution is a fundamental operation in image processing, commonly used for filtering, edge detection, sharpening, blurring, and feature extraction. It involves sliding a small matrix (called a kernel or filter) over an image and computing the sum of element-wise multiplications between the kernel and the corresponding region of the image.

$$g = h \star i  \quad  \rightarrow \quad g[x,y] = \sum_{u=-k}^{k}\sum_{v=-k}^{k}h[u,v]\,i[x-u,y-v]$$

where:
- $i$ is the image
- $h$ is the kernel
- $g(x,y)$ is the output pixel value at position $(x,y)$


**Padding** refer to adding extra pixels around the image to control how much the output shrinks after convolution. It is used to preserve the image size after the convolution.

Types of padding:
- Valid padding --> no padding, no extra pixels added
- Same padding --> extra pixels added to preserve the image size


**Stride** controls how much the kernel moves at each step when scanning the image (if 1, the kernel moves one pixel after another).

In [ ]:
# define a kernel (Sobel filter)
kernel = np.array([[-1, 0, 1],
                   [-2, 0, 2],
                   [-1, 0, 1]])

# apply convolution without padding (valid padding) (stride supposed to be 1)
output_valid = cv2.filter2D(image, -1, kernel)
output_valid = output_valid[1:-1, 1:-1]  # crop edges manually to match the expected size

# apply convolution with zero padding (same padding) (stride supposed to be 1)
padded_image = cv2.copyMakeBorder(image, 1, 1, 1, 1, cv2.BORDER_CONSTANT, value=0)  # adds border
output_same = cv2.filter2D(padded_image, -1, kernel)
output_same = output_same[1:-1, 1:-1]  # crop back to original size

original_size = image.shape
valid_size = output_valid.shape
padded_size = padded_image.shape
same_size = output_same.shape

plt.figure(figsize=(12, 6))

plt.subplot(1, 3, 1)
plt.imshow(image, cmap='gray')
plt.title(f'Original Image\n{original_size[1]}x{original_size[0]}')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(output_valid, cmap='gray')
plt.title(f'Without Padding (Valid)\n{valid_size[1]}x{valid_size[0]}')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(output_same, cmap='gray')
plt.title(f'With Padding (Same)\n{same_size[1]}x{same_size[0]}')
plt.axis('off')

plt.show()

**To compute the output size:**

$$W_{out} = \frac{(W-K+2P)}{S}+1$$
$$H_{out} = \frac{(H-K+2P)}{S}+1$$

where:
- $W$ is the original image width
- $H$ is the original image height
- $K$ is the kernel size
- $P$ is the padding ($P=0$ if valid padding, $P=\frac{(K-1)S}{2}$ if same padding)
- $S$ is the stride

Check with the above images!

# Image gradients

The image gradients detect and point to the areas where there is a rapid change in intensity, like edges.

$$∇_{xy}I = [\frac{\partial I[x,y]}{\partial x}, \frac{\partial I[x,y]}{\partial y}]$$

## Sobel filter and Derivative of Gaussian (DoG)

The Sobel filter is used to approximate the derivative of the image in the x and y directions

Horizontal Sobel filter (x direction)
$$
\frac{1}{8}
\begin{bmatrix}
-1 & 0 & 1 \\
-2 & 0 & 2 \\
-1 & 0 & 1
\end{bmatrix}
$$

Vertical Sobel filter (y direction)
$$
\frac{1}{8}
\begin{bmatrix}
1 & 2 & 1 \\
0 & 0 & 0 \\
-1 & -2 & -1
\end{bmatrix}
$$

<br>
The $\frac{1}{8}$ is needed to get the right gradient magnitude.

Derivative of Gaussian (DoG): Apply the Gaussian filter to the image to reduce the noise, and then compute the gradient of the image.

NOTE: the Sobel filter is the most common approximation of the DoG.

In [ ]:
# load the image in grayscale
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# compute the Sobel gradients in X and Y directions
sobel_x = cv2.Sobel(gray_image, cv2.CV_64F, 1, 0, ksize=3)  # derivative in x direction
sobel_y = cv2.Sobel(gray_image, cv2.CV_64F, 0, 1, ksize=3)  # derivative in y direction

# compute the magnitude and direction of the gradient
gradient_magnitude = cv2.magnitude(sobel_x, sobel_y)
gradient_direction = cv2.phase(sobel_x, sobel_y, angleInDegrees=True)

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.imshow(gradient_magnitude, cmap='gray')
plt.title('Derivative of Gaussian')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(gradient_direction, cmap='hsv')
plt.title('Gradient Direction')
plt.axis('off')

plt.show()

## Laplacian filter

It is a second derivative filter, working with the Laplacian operator:

$$∇^2I=\frac{∂^2I}{∂x^2}+\frac{∂^2I}{∂y^2}$$

In [ ]:
# apply the Laplacian filter
laplacian = cv2.Laplacian(gray_image, cv2.CV_64F)

# convert the result to uint8 (for visualization)
laplacian_abs = cv2.convertScaleAbs(laplacian)

plt.figure(figsize=(8, 8))

plt.imshow(laplacian_abs, cmap='gray')
plt.title('Laplacian Filter')
plt.axis('off')
plt.show()

## Laplacian of Gaussian (LoG)

Since the Laplacian is highly sensitive to noise, we apply a Gaussian filter to the image to reduce the noise and the Laplacian filter to detect edges.

In [ ]:
sigma = 1.0  # adjust sigma for different levels of smoothing
gaussian_blurred = cv2.GaussianBlur(gray_image, (5, 5), sigma)

laplacian_of_gaussian = cv2.Laplacian(gaussian_blurred, cv2.CV_64F)

# convert the result to uint8 (for visualization purposes)
log_result = cv2.convertScaleAbs(laplacian_of_gaussian)

plt.figure(figsize=(8, 8))
plt.imshow(log_result, cmap='gray')
plt.title('Laplacian of Gaussian')
plt.axis('off')

plt.show()

## Canny-edge detector

Computer vision pipeline whose steps are:


1.   Filter the image with DoG
2.   Find the magnitude and orientation of the gradient
3.   Non-maximum suppression (Each pixel is compared to its two neighbors in the gradient direction, and weaker ones are suppressed to not consider noisy pixels)
4.   Define two thresholds: low and high
5.   Use the high threshold to start edge curves and the low threshold to
     continue them

In [ ]:
low_threshold = 100  # lower threshold for edge detection
high_threshold = 300  # upper threshold for edge detection, this should be 3 times the low_threshold
edges = cv2.Canny(gray_image, low_threshold, high_threshold)

plt.figure(figsize=(8, 8))
plt.imshow(edges, cmap='gray')
plt.title('Canny Edge Detection')
plt.axis('off')

plt.show()

## Summary of edge detection methods:



- **DoG**: medium noise reduction, medium accuracy, low complexity
- **Laplacian**: low noise reduction, low accuracy, low complexity
- **LoG**: high noise reduction, high accuracy, medium complexity
- **Canny**: very high noise reduction, very high accuracy, high complexity

## Harris corner detector

Corner detection technique consisting of shifting a predefined window over the image. Such a window detects significant changes in all directions.

Steps:
- Compute Gaussian derivatives at each pixel
- Compute the second moment matrix H in a Gaussian window around each pixel
- Compute the corner response function
- Find local maxima of response function (non-maximum suppression)


Parameters of the cornerharris function:
- gray_float: gray scale image
- blockSize: size of neighbourhood considered for corner detection
- ksize: Sobel kernel size for the Gaussian derivatives
- k: is a free parameter. the lower, the more corners are detected with lower accuracy. The higher, the less corners are detected with higher accuracy

In [ ]:
image = cv2.imread("image.jpg")  # Replace with your image path
b,g,r = cv2.split(image)
image = cv2.merge([r,g,b])

gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray_float = np.float32(gray_image)

# apply Harris Corner Detector
harris_response = cv2.cornerHarris(gray_float, blockSize=2, ksize=3, k=0.04)

# threshold the response to identify strong corners
threshold = 0.01 * harris_response.max()
image[harris_response > threshold] = [0, 0, 255]  # Mark corners in blues

plt.figure(figsize=(10, 5))
plt.imshow(image)
plt.title("Harris Corner Detection")
plt.axis("off")
plt.show()

Properties of Harris-corner detector:
- equivariant to translation and rotation. ✅
- not equivariant to scaling. ❌

### Example with rotated image

In [ ]:
image = cv2.imread("image.jpg")  # Replace with your image path
b,g,r = cv2.split(image)
image = cv2.merge([r,g,b])

# image rotation
(h, w) = image.shape[:2]
center = (w // 2, h // 2)  # image center
angle = 45  # rotation angle
scale = 1.0  # we can scale if we want

rotation_matrix = cv2.getRotationMatrix2D(center, angle, scale)

# apply the rotation
rotated_image = cv2.warpAffine(image, rotation_matrix, (w, h))

gray_rotated = cv2.cvtColor(rotated_image, cv2.COLOR_RGB2GRAY)
gray_float_rotated = np.float32(gray_rotated)

harris_response_rotated = cv2.cornerHarris(gray_float_rotated, blockSize=2, ksize=3, k=0.04)

threshold = 0.01 * harris_response_rotated.max()
rotated_image[harris_response_rotated > threshold] = [0, 0, 255]

plt.figure(figsize=(10, 5))
plt.imshow(rotated_image)
plt.title("Harris Corner Detection on Rotated Image")
plt.axis("off")
plt.show()

### Example with scaled image

In [ ]:
image = cv2.imread("image.jpg")
b,g,r = cv2.split(image)
image = cv2.merge([r,g,b])

# scale the image
scale_factor = 0.2  # scale factor
new_size = (int(image.shape[1] * scale_factor), int(image.shape[0] * scale_factor))
scaled_image = cv2.resize(image, new_size, interpolation=cv2.INTER_LINEAR)

gray_scaled = cv2.cvtColor(scaled_image, cv2.COLOR_RGB2GRAY)
gray_float_scaled = np.float32(gray_scaled)

harris_response_scaled = cv2.cornerHarris(gray_float_scaled, blockSize=2, ksize=3, k=0.04)

threshold = 0.01 * harris_response_scaled.max()
scaled_image[harris_response_scaled > threshold] = [0, 0, 255]

plt.figure(figsize=(10, 5))
plt.imshow(scaled_image)
plt.title("Harris Corner Detection on Scaled Image")
plt.axis("off")
plt.show()

## Brief introduction to image pyramids

Image pyramids are algorithms consisting of processing and scaling the given image, so to get a sort of pyramid.

The pyramid is made up of octaves, and each octave is made up of a certain number of differently blurred versions (different sigmas) of the given image.

images belonging to the same octave have the same scale. The upper we go, the higher is the image scaled.

![Description](https://docs.opencv.org/4.x/Pyramids_Tutorial_Pyramid_Theory.png)

There are two main image Pyramids: Gaussian pyramid (for feature detection and matching) and Laplacian pyramid (for image reconstruction)

Gaussian pyramid algorithm.
- repeat until a minimum resolution is reached
  - apply the Gaussian filter
  - subsample the obtained image

Laplacian pyramid algorithm.
- repeat until a minimum resolution is reached
  - apply the Gaussian filter
  - compute the residual between the blurred images of step $t$ and $t-1$
  - subsample the obtained image

In [ ]:
image = cv2.imread('image.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# number of levels in the pyramid
num_levels = 4

# construct the Gaussian Pyramid
gaussian_pyramid = [image]  # start with the original image
for i in range(num_levels):
    image = cv2.pyrDown(image)  # downsample using Gaussian filter
    gaussian_pyramid.append(image)

# construct the Laplacian Pyramid
laplacian_pyramid = []
for i in range(num_levels, 0, -1):
    gaussian_expanded = cv2.pyrUp(gaussian_pyramid[i])  # upsample to match previous level
    gaussian_expanded = cv2.resize(gaussian_expanded, (gaussian_pyramid[i-1].shape[1], gaussian_pyramid[i-1].shape[0]))  # match size
    laplacian = cv2.subtract(gaussian_pyramid[i-1], gaussian_expanded)  # compute Laplacian
    laplacian_pyramid.append(laplacian)

fig, axes = plt.subplots(2, num_levels + 1, figsize=(15, 6))

for i in range(num_levels + 1):
    axes[0, i].imshow(gaussian_pyramid[i])
    axes[0, i].set_title(f'Gaussian {i}')
    axes[0, i].axis('off')

for i in range(num_levels):
    axes[1, i].imshow(laplacian_pyramid[i], cmap='gray')
    axes[1, i].set_title(f'Laplacian {i}')
    axes[1, i].axis('off')

axes[1, -1].axis('off')

plt.subplots_adjust(wspace=0.1, hspace=0.2)

plt.show()


# Fourier series

## Visualizing Fourier's Claim

The **fundamental claim** of Fourier is:
> *Any periodic signal can be represented as the sum of multiple sinusoids with different frequencies, amplitudes, and phases.*

Translated into mathematical terms, these **elementary blocks** are:

$$A \sin(2\pi f x + \phi)$$

Where:
- **Amplitude** ($A$): how strong the wave is;
- **Frequency** ($f$): how fast it oscillates;
- **Variable** ($x$): time or space;
- **Phase** ($\phi$): how shifted the wave is.

By combining enough of these components, we can approximate any periodic function, no matter how complex.


In [ ]:
# Signal definition in time domain
duration = 2.5 # [s]
sampling_rate = 1000  # [Hz]
N = int(duration * sampling_rate)  # Number of samples
t = np.linspace(0, duration, N, endpoint=False)

# Frequency components definition
frequencies = [3, 5, 7, 11]  # [Hz]
amplitudes = [1.0, 0.8, 0.6, 0.4]  # Amplitudes
phases = [0, np.pi/4, np.pi/2, np.pi]  # Initial phases [rad]
colors = ['red', 'blue', 'green', 'purple']  # Colors for each component

# Composite signal
sinusoids = [A * np.sin(2 * np.pi * f * t + p) for A, f, p in zip(amplitudes, frequencies, phases)]
signal = sum(sinusoids)

# 3D plot
fig = go.Figure()
time_extension = 1.1  # Visualization problem solver :)

# Add sinusoids components in the plot
for i, (f, sinusoid, color) in enumerate(zip(frequencies, sinusoids, colors)):
    fig.add_trace(
        go.Scatter3d(
          x = t,
          y = np.full_like(t, f),
          z = sinusoid,
          mode = 'lines',
          name = f'Sinusoid {f} Hz',
          line = dict(color = color)
        )
    )

# Add composite signal in the plot
fig.add_trace(
    go.Scatter3d(
        x = t,
        y = np.full_like(t, 0),
        z = signal,
        mode = 'lines',
        name = 'Composite Signal',
        line = dict(width = 4, color = 'orange')
    )
)

# Add frequency compentents in the plot
for i, (f, color, amp) in enumerate(zip(frequencies, colors, amplitudes)):
    # Frequency point
    fig.add_trace(
        go.Scatter3d(
            x = [duration * time_extension],
            y = [f],
            z = [amp],
            mode = 'markers',
            name = f'Frequency {f} Hz',
            marker = dict(size = 5, color = color)
        )
    )

    # Vertical bar connecting dots to frequency axis
    fig.add_trace(
        go.Scatter3d(
            x = [duration * time_extension, duration * time_extension],
            y = [f, f],
            z = [0, amp],
            mode = 'lines',
            line = dict(color = color, width = 3),
            showlegend = False
        )
    )

# Add a reference dashed axis for zero amplitude in frequency domain
fig.add_trace(
    go.Scatter3d(
        x = [duration * time_extension, duration * time_extension],
        y = [0, max(frequencies) + 2],
        z = [0, 0],
        mode = 'lines',
        line = dict(color = 'black', width = 2, dash = 'dash'),
        name = 'Zero Amplitude Axis'
    )
)

# Final figure layout
fig.update_layout(
    scene = dict(
        xaxis_title = "Time [s]",
        yaxis_title = "Frequency [Hz]",
        zaxis_title = "Amplitude",
        camera = dict(eye = dict(x = 1.5, y = 1.5, z = 1.0))
    ),
    title = "Time Signal Frequency Decomposition",
    width = 1400,
    height = 950
)

fig.show(renderer = "colab")

In [ ]:
def fourier_approximation(t, num_components=5, signal_type='square'):
    """
    Calculate the Fourier series approximation for different types of signals
    ensuring exactly num_components are returned
    """
    y = np.zeros_like(t)
    components = []
    frequencies = []

    if signal_type == 'square':
        """
          Square wave: sum of odd sinusoids with amplitude 1/n,
          so we need to count only odd harmonics
        """
        n = 1
        count = 0
        while count < num_components:
            term = (4/np.pi) * (1/n) * np.sin(n * t)
            y += term
            components.append(term)
            frequencies.append(n)
            count += 1
            n += 2
        target = np.sign(np.sin(t))

    elif signal_type == 'sawtooth':
        """
          Sawtooth wave: all harmonics with alternating signs and amplitude 1/n
        """
        for n in range(1, num_components + 1):
            term = (2/np.pi) * ((-1)**(n+1) / n) * np.sin(n * t)
            y += term
            components.append(term)
            frequencies.append(n)
        target = 2 * (t/(2*np.pi) - np.floor(t/(2*np.pi) + 0.5))

    elif signal_type == 'triangle':
        """
          Triangle wave: sum of odd sinusoids with alternating signs and
          amplitude 1/n², so we need to count only odd harmonics
        """
        n = 1
        count = 0
        while count < num_components:
            term = (8/(np.pi**2)) * ((-1)**((n-1)//2) / (n**2)) * np.sin(n * t)
            y += term
            components.append(term)
            frequencies.append(n)
            count += 1
            n += 2
        target = 2 * np.abs((t - np.pi/2) % (2*np.pi) / np.pi - 1) - 1

    return y, components, frequencies, target

def plot_fourier_series(num_components=5, signal_type='square'):
    t = np.linspace(0, 4 * np.pi, 1000) # [s]
    y, components, frequencies, target = fourier_approximation(t, num_components, signal_type)

    # Figure
    fig = plt.figure(figsize=(18, 8))
    gs = GridSpec(1, 3, width_ratios=[1, 1, 1])

    # First plot: Sum of components (approximation) vs. target signal
    ax1 = fig.add_subplot(gs[0])
    ax1.plot(t, target, 'k--', label='Target signal')
    ax1.plot(t, y, 'r-', linewidth=2, label=f'Approximation (n={num_components})')
    ax1.set_title('Fourier Series Approximation')
    ax1.set_xlabel('Time (s)')
    ax1.set_ylabel('Amplitude')
    ax1.grid(True)
    ax1.set_xlim(0, 4 * np.pi)
    ax1.legend()

    # Second plot: Individual sinusoidal components
    ax2 = fig.add_subplot(gs[1])
    component_height = 1.0 / len(components)

    for i, component in enumerate(components):
        # Scale the component for better visualization
        scaled_component = component * 0.4 * component_height

        # Position each component in a space in the plot
        position = 1.0 - (i + 0.5) * component_height
        ax2.plot(t, scaled_component + position, label=f'Component {frequencies[i]}')

        # Add a horizontal line to separate components
        ax2.axhline(y=1.0 - (i+1) * component_height, color='gray', linestyle='-', alpha=0.3)

    ax2.set_title(f'Sinusoidal Components ({len(components)} shown)')
    ax2.set_xlabel('Time (s)')
    ax2.set_yticks([])
    ax2.set_xlim(0, 4 * np.pi)
    ax2.grid(True)

    # Third plot: Frequency spectrum (amplitude vs. frequency)
    ax3 = fig.add_subplot(gs[2])

    # Calculate amplitudes for each frequency
    amplitudes = []
    for i, freq in enumerate(frequencies):
        if signal_type == 'square':
            amplitudes.append((4/np.pi) * (1/freq))
        elif signal_type == 'sawtooth':
            amplitudes.append((2/np.pi) * ((-1)**(freq+1) / freq))
        elif signal_type == 'triangle':
            amplitudes.append((8/(np.pi**2)) * ((-1)**((freq-1)//2) / (freq**2)))

    # Display the discrete spectrum
    ax3.grid(True, zorder=0)
    ax3.bar(frequencies, np.abs(amplitudes), alpha=0.8, color='blue', zorder=3)
    ax3.set_title('Frequency Spectrum')
    ax3.set_xlabel('Frequency (Hz)')
    ax3.set_ylabel('Amplitude')
    ax3.set_xticks(frequencies)

    plt.tight_layout()
    plt.show()

# Interactive widgets to control parameters
@interact
def interactive_fourier(
    num_components=IntSlider(min=1, max=20, step=1, value=5, description='Components:'),
    signal_type=widgets.Dropdown(
        options=[('Square wave', 'square'), ('Sawtooth wave', 'sawtooth'), ('Triangle wave', 'triangle')],
        value='square',
        description='Signal type:'
    )
):
    plot_fourier_series(num_components, signal_type)

## Filtering signals
In the frequency representation of a signal, each component carries specific information:

- **Low frequencies** represent the **smooth, global structure** of the signal.
- **High frequencies** capture the **sharp details and rapid variations**.

This means that by **filtering** out:
  - High frequencies, we remove details and retain the general shape of the signal.
  - Low frequencies, we remove the base structure and emphasizes fine details

In [ ]:
def fourier_approximation_filtered(t, num_components=5, signal_type='square', filter_range=(0, 20)):
    y = np.zeros_like(t)
    components = []
    frequencies = []

    if signal_type == 'square':
        """
          Square wave: sum of odd sinusoids with amplitude 1/n,
          so we need to count only odd harmonics
        """
        n = 1
        count = 0
        while count < num_components:
            term = (4/np.pi) * (1/n) * np.sin(n * t)
            if filter_range[0] <= n <= filter_range[1]:
                y += term
            components.append(term)
            frequencies.append(n)
            count += 1
            n += 2
        target = np.sign(np.sin(t))

    elif signal_type == 'sawtooth':
        """
          Sawtooth wave: all harmonics with alternating signs and amplitude 1/n
        """
        n = 1
        count = 0
        while count < num_components:
            term = (2/np.pi) * ((-1)**(n+1) / n) * np.sin(n * t)
            if filter_range[0] <= n <= filter_range[1]:
                y += term
            components.append(term)
            frequencies.append(n)
            count += 1
            n += 1
        target = 2 * (t/(2*np.pi) - np.floor(t/(2*np.pi) + 0.5))

    elif signal_type == 'triangle':
        """
          Triangle wave: sum of odd sinusoids with alternating signs and
          amplitude 1/n², so we need to count only odd harmonics
        """
        n = 1
        count = 0
        while count < num_components:
            term = (8/(np.pi**2)) * ((-1)**((n-1)//2) / (n**2)) * np.sin(n * t)
            if filter_range[0] <= n <= filter_range[1]:
                y += term
            components.append(term)
            frequencies.append(n)
            count += 1
            n += 2
        target = 2 * np.abs((t - np.pi/2) % (2*np.pi) / np.pi - 1) - 1

    return y, components, frequencies, target, filter_range

def plot_fourier_series_filtered(num_components=5, signal_type='square', filter_range=(0, 20)):
    t = np.linspace(0, 4 * np.pi, 1000) # [s]
    y, components, frequencies, target, filter_range = fourier_approximation_filtered(t, num_components, signal_type, filter_range)

    # Figure
    fig = plt.figure(figsize=(18, 8))
    gs = GridSpec(1, 3, width_ratios=[1, 1, 1])

    # First plot: Sum of components (approximation) vs. target signal
    ax1 = fig.add_subplot(gs[0])
    ax1.plot(t, target, 'k--', label='Target signal')
    ax1.plot(t, y, 'r-', linewidth=2, label=f'Filtered approx. (n={num_components})')
    ax1.set_title('Fourier Series Approximation')
    ax1.set_xlabel('Time (s)')
    ax1.set_ylabel('Amplitude')
    ax1.grid(True)
    ax1.set_xlim(0, 4 * np.pi)
    ax1.legend()

    # Second plot: Individual sinusoidal components
    ax2 = fig.add_subplot(gs[1])
    component_height = 1.0 / max(len(components), 1)

    for i, (component, freq) in enumerate(zip(components, frequencies)):
        # Scale the component for better visualization
        scaled_component = component * 0.4 * component_height

        # Position each component in a space in the plot
        position = 1.0 - (i + 0.5) * component_height

        # Determine if this component is filtered out
        is_filtered = filter_range[0] <= freq <= filter_range[1]

        # Plot component
        if is_filtered:
            ax2.plot(t, scaled_component + position, label=f'Component {freq}')
        else:
            # Gray out filtered components
            ax2.plot(t, scaled_component + position, color='gray', alpha=0.4, label=f'Component {freq} (filtered)')

        # Add a horizontal line to separate components
        ax2.axhline(y=1.0 - (i+1) * component_height, color='gray', linestyle='-', alpha=0.3)

    ax2.set_title(f'Sinusoidal Components ({len(components)} shown)')
    ax2.set_xlabel('Time (s)')
    ax2.set_yticks([])
    ax2.set_xlim(0, 4 * np.pi)
    ax2.grid(True)

    # Third plot: Frequency spectrum (amplitude vs. frequency) with filter visualization
    ax3 = fig.add_subplot(gs[2])

    # Calculate amplitudes for each frequency
    amplitudes = []
    for i, freq in enumerate(frequencies):
        if signal_type == 'square':
            amplitudes.append((4/np.pi) * (1/freq))
        elif signal_type == 'sawtooth':
            amplitudes.append((2/np.pi) * ((-1)**(freq+1) / freq))
        elif signal_type == 'triangle':
            amplitudes.append((8/(np.pi**2)) * ((-1)**((freq-1)//2) / (freq**2)))

    # Display the discrete spectrum with filter visualization
    ax3.grid(True, zorder=0)

    # Draw out-filter region as a light blue rectangle
    max_amplitude = max(np.abs(amplitudes)) if amplitudes else 1
    filter_height = max_amplitude * 1.2
    rect = plt.Rectangle((0, 0), max(frequencies) + 1, filter_height,
                         facecolor='lightblue', alpha=0.3, zorder=1)
    ax3.add_patch(rect)

    # Draw the filter cut-off regions
    left_rect = plt.Rectangle((0, 0), filter_range[0], filter_height,
                             facecolor='red', alpha=0.2, zorder=2)
    right_rect = plt.Rectangle((filter_range[1], 0), max(frequencies) + 1 - filter_range[1], filter_height,
                              facecolor='red', alpha=0.2, zorder=2)
    ax3.add_patch(left_rect)
    ax3.add_patch(right_rect)

    # Plot spectrum bars with appropriate colors based on filtering
    for i, (freq, amp) in enumerate(zip(frequencies, amplitudes)):
        if filter_range[0] <= freq <= filter_range[1]:
            ax3.bar(freq, np.abs(amp), alpha=0.8, color='blue', zorder=3)
        else:
            # Gray out filtered frequencies
            ax3.bar(freq, np.abs(amp), alpha=0.4, color='gray', zorder=3)

    ax3.set_title('Frequency Spectrum with Filter')
    ax3.set_xlabel('Frequency (Hz)')
    ax3.set_ylabel('Amplitude')
    ax3.set_xlim(0, max(frequencies) + 1)
    ax3.set_xticks(frequencies)

    plt.tight_layout()
    plt.show()

# Interactive widgets to control parameters
@interact
def interactive_fourier_filtered(
    num_components=IntSlider(min=1, max=20, step=1, value=5, description='Components:'),
    signal_type=widgets.Dropdown(
        options=[('Square wave', 'square'), ('Sawtooth wave', 'sawtooth'), ('Triangle wave', 'triangle')],
        value='square',
        description='Signal type:'
    ),
    filter_range=FloatRangeSlider(
        value=[0, 50],
        min=0,
        max=50,
        step=1,
        description='Filter [Hz]:',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='.0f',
    )
):
    plot_fourier_series_filtered(num_components, signal_type, filter_range)

## Let's Exaggerate!

To further explore how Fourier series work, let’s take a look at some **more extreme applications**:

🔗 **[Fourier Epicycles](https://www.myfourierepicycles.com/)** – A visualization showing how Fourier series can reconstruct **complex drawings** using rotating circles.  
🎥 **[Fourier Series Visualization](https://www.youtube.com/watch?v=r6sGWTCMz2k)** – A fantastic animation demonstrating how sinusoids combine to form intricate shapes (until drawing Fourier itself 🙂)


## Why was 1D Important?

Everything we have explored in **1D Fourier series** forms the foundation for understanding **2D Fourier Transform** in future applications.

- In **1D**, we analyzed how different frequencies contribute to a **signal**.
- We saw that **low frequencies** define the overall shape, while **high frequencies** capture finer details.
- **Filtering in 1D** helped us understand how removing specific frequencies affects the reconstruction.

## Before Moving On: The True Fourier Series

So far, we have introduced the **Fourier Series** conceptually as a sum of elementary blocks of the form:

$$ A \sin(2\pi f t + \phi) $$

This representation helped us understand that a periodic signal is **composed of sinusoids** with different frequencies, amplitudes, and phases.

However, this is not the **canonical mathematical expression** of the Fourier Series. The true formulation is:

$$ s(t) = \frac{a_0}{2} + \sum_{k=1}^{\infty} \left( a_k \cos(2\pi f k t) + b_n \sin(2\pi f k t) \right) $$

Where:
- $ \frac{a_0}{2} $ represents the average value of the function over the period $T$.
- $ a_k $ and $ b_k $ are the **Fourier coefficients**, which determine how much each cosine and sine component contributes.
- $ f = \frac{1}{T} $ is the **fundamental frequency** of the signal.
- $ k $ is the **harmonic index**, meaning each term corresponds to a multiple of the fundamental frequency.

This formula expresses any periodic function as a combination of sine and cosine waves (the sinusoids of the Fourier's claim), whose frequencies are determined by the function's period.

The Fourier Series can be expressed also in complex form:
$$ s(t) = \sum_{k=-\infty}^{\infty} c_n e^{j 2\pi k f t} $$

Where:
- $ c_n $ are the **complex coefficients** that contain information about the amplitude and phase of each frequency component.

# Fourier Transform

## A Transformation, Unbound by Time

The **Fourier Transform** allows us to obtain the **frequency representation** of any signal, revealing:
- **Which frequencies are present** in the signal.
- **How strong** each frequency component is.

In mathematical terms, it converts a signal from the **time domain** to the **frequency domain**, giving us a complete view of its spectral composition.

$$ \mathcal{F} \{ f(t) \} = F(f) = \int_{-\infty}^{+\infty} f(t) e^{-j 2\pi f t} dt $$

The **Fourier Transform** output is a function in a **complex domain**. To recover the original real-valued signal, we use the **Inverse Fourier Transform**:

$$ \mathcal{F} \{ F(f) \}^{-1} = f(t) = \int_{-\infty}^{+\infty} F(f) e^{j 2\pi f t} df $$

The **symmetry properties** of the Fourier Transform ensure that when applying the inverse transform, the result is real-valued (if the original function was real).

Thus, the Fourier Transform serves as a bridge between real-world signals and the complex domain of frequency analysis, allowing operations to be performed across different domains. This proves particularly useful in certain contexts.

<div align="center">
    <img src="https://drive.google.com/uc?export=download&id=11xYvGceXhcDCFWdG_T7O2WGKHN9UMtmd" width="500">
</div>

## Beyond Periodicity

We have seen how the **Fourier Series** allows us to express periodic signals as sums of sinusoids with different frequencies, amplitudes, and phases. But what if the signal is **not periodic**?

This is where the **Fourier Transform** comes in.

With that tool, it is possible to "update" the previous Fourier claim:
> *Any signal, periodic or not, can be represented in terms of its frequency components using the Fourier Transform.*



In [ ]:
def generate_signal(fs=1000, N=1000, f1=50, f2=120, f3=200, a1=1.0, a2=0.5, a3=0.75, noise_level=0.2):
    T = 1 / fs  # Sampling period
    t = np.linspace(0, (N - 1) * T, N)  # Time vector

    # Create the signal as the sum of three sinusoids
    signal = a1 * np.sin(2 * np.pi * f1 * t) + \
             a2 * np.sin(2 * np.pi * f2 * t) + \
             a3 * np.sin(2 * np.pi * f3 * t)

    # Add Gaussian noise
    np.random.seed(42)
    noise = noise_level * np.random.normal(0, 1, N)
    noisy_signal = signal + noise

    return t, noisy_signal, [(f1, a1), (f2, a2), (f3, a3)]

def compute_fft(signal, fs, N):
    yf = fft(signal)
    xf = fftfreq(N, 1/fs)[:N//2]
    amplitude_spectrum = 2.0 / N * np.abs(yf[:N//2])

    return xf, amplitude_spectrum

def plot_signal_analysis(fs=1000, N=1000, f1=50, f2=120, f3=200, a1=1.0, a2=0.5, a3=0.75, noise_level=0.2):
    t, noisy_signal, true_values = generate_signal(fs, N, f1, f2, f3, a1, a2, a3, noise_level)
    xf, amplitude_spectrum = compute_fft(noisy_signal, fs, N)

    # Identify peak frequencies
    threshold = 0.3  # Threshold for detecting peaks
    detected_frequencies = [(freq, round(amp, 3)) for freq, amp in zip(xf, amplitude_spectrum) if amp > threshold]

    # Figure
    fig = plt.figure(figsize=(18, 6))
    fig.suptitle(
        "Fourier Transform of a Noisy Signal Composed of Multiple Periodic Components",
        weight = "bold",
        fontsize = 16
    )
    gs = GridSpec(1, 2, width_ratios=[1, 1])

    # Time-domain plot
    ax1 = fig.add_subplot(gs[0])
    ax1.plot(t, noisy_signal, color='b')
    ax1.set_title('Time Domain')
    ax1.set_xlabel('Time [s]')
    ax1.set_ylabel('Amplitude')
    ax1.grid(True)

    # Frequency spectrum plot
    ax2 = fig.add_subplot(gs[1])
    ax2.plot(xf, amplitude_spectrum, color='r')
    ax2.set_title('Frequency Spectrum')
    ax2.set_xlabel('Frequency [Hz]')
    ax2.set_ylabel('Amplitude')
    ax2.grid(True)
    ax2.set_xlim(0, fs/2)

    plt.tight_layout()
    plt.show()

    # Table for expected vs detected values
    table = PrettyTable(
        [
            "\033[1mExpected Frequency (Hz)\033[0m",
            "\033[1mExpected Amplitude\033[0m",
            "\033[1mDetected Frequency (Hz)\033[0m",
            "\033[1mDetected Amplitude\033[0m"
        ]
    )
    expected_freqs, expected_amps = zip(*true_values)
    detected_freqs, detected_amps = zip(*detected_frequencies) if detected_frequencies else ([], [])

    for i in range(len(expected_freqs)):
        detected_freq = detected_freqs[i] if i < len(detected_freqs) else 'N/A'
        detected_amp = detected_amps[i] if i < len(detected_amps) else 'N/A'
        table.add_row([expected_freqs[i], expected_amps[i], detected_freq, detected_amp])

    print(table)

plot_signal_analysis()

In [ ]:
def generate_time_vector(fs=1000, N=1000):
    T = 1 / fs  # Sampling period
    return np.arange(-N//2, N//2) * T

def compute_fft(signal, fs, N):
    yf = fftshift(fft(signal))
    xf = fftshift(fftfreq(N, 1/fs))
    amplitude_spectrum = np.abs(yf) / N
    return xf, amplitude_spectrum

def plot_signal_and_fft(signal, time_title, freq_title, t, fs, N, ax_time, ax_freq):
    xf, amplitude_spectrum = compute_fft(signal, fs, N)

    # Time-domain plot
    ax_time.plot(t, signal, color='b', linewidth=1.5)
    ax_time.set_title(time_title)
    ax_time.set_xlabel('Time [s]')
    ax_time.set_ylabel('Amplitude')
    ax_time.grid(True)

    # Frequency-domain plot
    ax_freq.plot(xf, amplitude_spectrum, color='r', linewidth=1.5)
    ax_freq.set_title(freq_title)
    ax_freq.set_xlabel('Frequency [Hz]')
    ax_freq.set_ylabel('Amplitude')
    ax_freq.grid(True)

def plot_all_signals(fs=1000, N=1000):
    t = generate_time_vector(fs, N)

    # Define signals
    rect_width = 0.1
    rect_signal = np.zeros_like(t)
    rect_signal[np.abs(t) < rect_width/2] = 1.0

    sigma = 0.05
    gaussian_signal = np.exp(-0.5 * (t/sigma)**2)

    alpha = 100
    step_signal = np.heaviside(t, 1)

    # Figure
    fig = plt.figure(figsize=(18, 9))
    fig.suptitle(
        "Fourier Transform of Canonical Aperiodic Signals",
        weight = "bold",
        fontsize = 16
    )
    gs = GridSpec(3, 2, width_ratios=[1, 1])

    ax1 = fig.add_subplot(gs[0, 0])
    ax2 = fig.add_subplot(gs[0, 1])
    ax3 = fig.add_subplot(gs[1, 0])
    ax4 = fig.add_subplot(gs[1, 1])
    ax5 = fig.add_subplot(gs[2, 0])
    ax6 = fig.add_subplot(gs[2, 1])

    plot_signal_and_fft(
        rect_signal,
        "Rectangular Pulse - Time Domain",
        "Sinc Function - Frequency Domain",
        t, fs, N, ax1, ax2
    )
    plot_signal_and_fft(
        gaussian_signal,
        "Gaussian Pulse - Time Domain",
        "Gaussian Function - Frequency Domain",
        t, fs, N, ax3, ax4
    )
    plot_signal_and_fft(
        step_signal,
        "Step Function - Time Domain",
        "Transformed Step - Frequency Domain",
        t, fs, N, ax5, ax6
    )

    plt.tight_layout()
    plt.show()

plot_all_signals()


## Fourier Series vs Fourier Transform: The (Student's) inevitable Question

1. **The Fourier Series is a special case of the Fourier Transform.**  
   - The Fourier Series applies **only to periodic signals**, producing a **discrete spectrum**.
   - The Fourier Transform applies to **any signal**, producing a **continuous spectrum**.

2. **They are NOT inverse operations!**  
   - The Fourier Series is not the inverse of the Fourier Transform.

\\

|  | **Fourier Series** | **Fourier Transform** |
|---|------------------|--------------------|
| **Input** | **Periodic signal in time domain** $ s(t) $ | **Any signal in time domain** $ f(t) $ |
| **Output** | **Periodic function in time domain** (sum of sinusoids) | **Complex function in frequency domain** $ F(f) $ |
| **Definition** | Represents a **periodic signal** as a sum of sinusoids at harmonic frequencies | Decomposes **any signal** into its frequency components |
| **Frequencies** | **Discrete** (multiples of a fundamental frequency) | **Continuous** (any frequency value) |
| **Formula** | $$ s(t) = \sum_{k=-\infty}^{\infty} c_k e^{j 2\pi k f t} $$ | $$ F(f) = \int_{-\infty}^{+\infty} f(t) e^{-j 2\pi f t} dt $$ |

\\

# Discrete Fourier Transform



## From Continuous to Computable
The **Fourier Transform** is a powerful tool for analyzing signals in the **frequency domain**. However, in practical applications—especially in **computer vision, audio processing, and numerical computing**—we often deal with **discrete** data rather than continuous functions.

This is where the **Discrete Fourier Transform (DFT)** comes into play.

The DFT is the discrete counterpart of the Fourier Transform, defined for a **finite sequence** of $ N $ samples:

$$ X_k = \sum_{n=0}^{N-1} x_n e^{\frac{-j 2\pi k n}{N}}, \quad k = 0, 1, ..., N-1 $$

Where:
- $ x_n $ represents the sample of input signal in the **time domain**.
- $ X_k $ represents the sample of output in the **frequency domain**.
- $ N $ is the number of discrete samples.
- $ k $ is the frequency bin.
- The exponential term $ e^{-j 2\pi k n / N} $ represents the contribution of each frequency component.

In this case, we can interpret $ k/N $ as the **normalized frequency**, corresponding to the continuous frequency in the Fourier Transform. Similarly, $ n $ represents the **discrete time index**, which relates to the continuous time variable in the original signal.


Since real-world data is often **sampled** rather than continuous, the **DFT is essential** because **computers process discrete signals**, so we cannot apply the continuous Fourier Transform directly.

## Complex... but Simple!

The **Discrete Fourier Transform (DFT)** allows us to express a discrete signal in terms of its frequency components. However, before diving into its interpretation, it is crucial to recognize a fundamental fact:

### 1️⃣ **Each $X_k$ is Simply a Complex Number**
The DFT formula consists of a summation of weighted complex exponentials:

$$ X_k = \sum_{n=0}^{N-1} x_n e^{\frac{-j 2\pi k n}{N}} $$

Rewriting the exponent:

$$ X_k = \sum_{n=0}^{N-1} x_n e^{-j b_n} $$

Expanding the summation:

$$ X_k = x_0 e^{-j b_0} + x_1 e^{-j b_1} + ... + x_{N-1} e^{-jb_{N-1}} = A_k + j B_k $$

This means that **the output of the DFT at each $k$ is nothing more than a complex number**, composed of:
- A **real part** $A_k$.
- An **imaginary part** $B_k$.

---

### 2️⃣ **Representing $X_k$ in the Frequency Domain**
Since $X_k$ is a complex number, to visualize it in the **frequency domain**, we use the standard representation of complex numbers in terms of **magnitude** and **phase**:

- **Magnitude** (strength of the frequency component):

  $$ |X_k| = \sqrt{A_k^2 + B_k^2} $$

- **Phase** (how the frequency component is shifted):

  $$ \angle X_k = \theta $$

This conversion allows us to express each $X_k$ in a way that is **meaningful in the frequency domain**, rather than just as real and imaginary parts.

<div align="center">
    <img src="https://drive.google.com/uc?export=download&id=1-mIlEcSPJ_x74-646eAjkGFDiEFYLCQP" width="500">
</div>

---

### 3️⃣ **Repeating This Process for Every $k$**
By repeating this process for every $k$, we obtain **two plots** that fully describe the signal in the frequency domain:

1. **Magnitude Spectrum**: A plot of $|X_k|$, showing the contribution of each frequency.
2. **Phase Spectrum**: A plot of $\angle X_k$, indicating how each frequency component is shifted in time.

Together, these two plots provide **a complete representation of the signal's frequency content**, relative to its original sampled version in the time domain. Thus, the DFT does not output anything mysterious, just a sequence of complex numbers, whose magnitude and phase tell us how the frequencies combine to form the original signal.




In [ ]:
fs = 10  # Sampling frequency [Hz]
T = 2.0  # Duration [s]
N = int(T * fs)  # Number of samples
t = np.linspace(0, T, N, endpoint=False)  # Time

# Continuos signal
def f(t):
  return 5 + 2 * np.cos(2 * np.pi * t - np.pi/2) + 3 * np.cos(4 * np.pi * t)
signal = f(t)

# Calculate DFT
fft_result = fft(signal)
fft_freq = np.fft.fftfreq(N, 1/fs)

# Calculate magnitude and phase
magnitude = np.abs(fft_result) / N  # Normalization
phase = np.angle(fft_result, deg=True)

# Visualization stuff
positive_freq_indices = np.where(fft_freq >= 0)[0]
min_display_magnitude = 0.01
for i in range(len(magnitude)):
    if magnitude[i] < min_display_magnitude:
        magnitude[i] = min_display_magnitude

def plot_fft(freq_idx=0):
    # Figure
    plt.clf()
    fig = plt.figure(figsize=(18, 10))
    gs = GridSpec(2, 2)

    # First plot: original continuous and sampled function
    ax1 = fig.add_subplot(gs[0, 0])
    t_continuous = np.linspace(0, T, 1000)
    signal_continuous = f(t_continuous)
    ax1.plot(t_continuous, signal_continuous, 'b-', linewidth=2, label='Continuous Signal')
    ax1.plot(t, signal, 'ro', markersize=8, label='Samples')
    ax1.set_xlabel('Time (s)')
    ax1.set_ylabel('Amplitude')
    ax1.set_title('Original Signal f(t) and Samples')
    ax1.grid(True)
    ax1.legend()

    # Second plot: DFT magnitude
    ax2 = fig.add_subplot(gs[0, 1])
    markerline, stemlines, baseline = ax2.stem(
        fft_freq[positive_freq_indices],
        magnitude[positive_freq_indices],
        'purple',
        markerfmt='mo',
        basefmt=" "
    )
    plt.setp(stemlines, 'linewidth', 2)

    # Highlight selected frequency
    selected_idx = positive_freq_indices[freq_idx]
    ax1.plot(t[selected_idx], signal[selected_idx], 'yo', markersize=10)
    ax2.plot(fft_freq[selected_idx], magnitude[selected_idx], 'yo', markersize=10)

    ax2.set_xlabel('Frequency (Hz)')
    ax2.set_ylabel('Amplitude')
    ax2.set_title('DFT Magnitude')
    ax2.set_xlim(-0.5, fs/2 + 0.5)
    ax2.grid(True)

    # Third plot: phase of DFT
    ax3 = fig.add_subplot(gs[1, 0])
    markerline, stemlines, baseline = ax3.stem(
        fft_freq[positive_freq_indices],
        phase[positive_freq_indices],
        'g', markerfmt='go', basefmt=" "  # Remove baseline
    )
    plt.setp(stemlines, 'linewidth', 2)

    # Highlight selected frequency
    ax3.plot(fft_freq[selected_idx], phase[selected_idx], 'yo', markersize=10)
    ax3.set_xlabel('Frequency (Hz)')
    ax3.set_ylabel('Phase (degrees)')
    ax3.set_title('DFT Phase')
    ax3.set_xlim(-0.5, fs/2 + 0.5)
    ax3.grid(True)

    # Fourth plot: complex plane
    ax4 = fig.add_subplot(gs[1, 1])
    ax4.set_aspect('equal')
    ax4.axhline(y=0, color='k', linestyle='-', alpha=0.3)
    ax4.axvline(x=0, color='k', linestyle='-', alpha=0.3)

    # Unit circle
    theta = np.linspace(0, 2*np.pi, 100)
    ax4.plot(np.cos(theta), np.sin(theta), 'k--', alpha=0.3)

    # Plot frequency components in complex plane
    for i, idx in enumerate(positive_freq_indices):
        re = magnitude[idx] * np.cos(np.radians(phase[idx]))
        im = magnitude[idx] * np.sin(np.radians(phase[idx]))
        ax4.plot([0, re], [0, im], 'b-', alpha=0.3)
        ax4.plot(re, im, 'bo', alpha=0.5)

    # Highlight selected component in complex plane
    re_selected = magnitude[selected_idx] * np.cos(np.radians(phase[selected_idx]))
    im_selected = magnitude[selected_idx] * np.sin(np.radians(phase[selected_idx]))
    ax4.plot([0, re_selected], [0, im_selected], 'y-', linewidth=2)
    ax4.plot(re_selected, im_selected, 'yo', markersize=10)

    freq_value = fft_freq[selected_idx]
    mag_value = magnitude[selected_idx]
    phase_value = phase[selected_idx]
    original_mag = np.abs(fft_result[selected_idx]) / N

    ax4.text(0.05, 0.95, f'Frequency: {freq_value:.2f} Hz', transform=ax4.transAxes)
    ax4.text(0.05, 0.9, f'Magnitude: {original_mag:.4f}', transform=ax4.transAxes)
    ax4.text(0.05, 0.85, f'Phase: {phase_value:.2f}°', transform=ax4.transAxes)

    ax4.set_xlabel('Real Part')
    ax4.set_ylabel('Imaginary Part')
    ax4.set_title('Complex Plane Representation')
    ax4.grid(True)

    max_mag = np.max(magnitude) * 1.2
    ax4.set_xlim(-max_mag, max_mag)
    ax4.set_ylim(-max_mag, max_mag)

    plt.tight_layout()
    plt.show()

# Interactive plot
slider = IntSlider(
    min=0,
    max=len(positive_freq_indices)-1,
    step=1,
    value=0,
    description='Sample:',
    continuous_update=False,
    layout={'width': '400px'}
)

print("Use the slider below to select different frequency components:")
interact(plot_fft, freq_idx=slider)

🎥 **For further insights:** [DFT Visualization](https://www.youtube.com/watch?v=mkGsMWi_j4Q)  

# Frequency domain for Computer Vision

## What Changes in 2D?

In **2D**, the same logic applies, but now we analyze **images** instead of 1D signals.  

A greyscale image can be thought of as a **2D discrete signal**, where:
- The **spatial dimensions** (rows and columns) correspond to **time samples** in the 1D case.
- Each pixel value represents an **amplitude**, similar to how a signal varies in time.

Unlike in the **1D case**, where we obtained a **1D magnitude and phase plot**, the **2D Discrete Fourier Transform (DFT)** produces:

- A **2D magnitude spectrum**, representing how much of each spatial frequency is present in the image.
  - **Bright areas** indicate **strong frequency components**, meaning there are prominent repeating patterns or edges in those spatial directions.
  - **Dark areas** represent **low energy**, meaning little variation in those frequencies.
- A **2D phase spectrum**, indicating how different frequency components are shifted spatially.
  - The phase contains information about **spatial shifts** in the image.
  - Unlike the magnitude, which is easier to interpret, the phase is more abstract but is crucial for reconstructing the original image.

In these graphs, the low frequencies are concentrated towards the center of the images, while the high frequencies are located towards the edges.

In [ ]:
def fft_analysis(image):
    f = np.fft.fft2(image)
    fshift = np.fft.fftshift(f)
    magnitude_spectrum = np.abs(fshift)
    phase_spectrum = np.angle(fshift)
    return fshift, magnitude_spectrum, phase_spectrum

def reconstruct_image(magnitude, phase):
    complex_spectrum = magnitude * np.exp(1j * phase)
    f_ishift = np.fft.ifftshift(complex_spectrum)
    img_reconstructed = np.fft.ifft2(f_ishift)
    return np.abs(img_reconstructed)

# Convert to grayscale
image_bw = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2GRAY)

# Compute image DFT
fshift, magnitude, phase = fft_analysis(image_bw)
image_mag = reconstruct_image(magnitude, np.zeros_like(phase))
image_phase = reconstruct_image(np.ones_like(magnitude), phase)

# Figure
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Original image
axes[0].imshow(image_bw, cmap='gray')
axes[0].set_title("Original Image")

# Magnitude spectrum
axes[1].imshow(np.log(1 + magnitude), cmap='viridis')
axes[1].set_title("Magnitude Spectrum")
axes[1].axis('off')

# Phase spectrum
axes[2].imshow(phase, cmap='viridis')
axes[2].set_title("Phase Spectrum")
axes[2].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
def dft_analysis(image):
    dft = np.fft.fft2(image)
    dft_shift = np.fft.fftshift(dft)
    magnitude = np.abs(dft_shift)
    phase = np.angle(dft_shift)
    return dft_shift, magnitude, phase

def scale_image(image, scale_factor):
    new_size = (int(image.shape[1] * scale_factor), int(image.shape[0] * scale_factor))
    scaled_image = cv2.resize(image, new_size, interpolation=cv2.INTER_CUBIC)
    return scaled_image

# Image DFT
dft_shift, magnitude, phase = dft_analysis(image_bw)

# Scaled image
scale_factor = 0.09
image_scaled = scale_image(image_bw, scale_factor)

# Scaled image DFT
dft_shift_scaled, magnitude_scaled, phase_scaled = dft_analysis(image_scaled)

# Figure
fig, axes = plt.subplots(2, 3, figsize=(15, 9))

# Original image
axes[0, 0].imshow(image_bw, cmap='gray')
axes[0, 0].set_title("Original Image")

# Image magnitude
axes[0, 1].imshow(np.log(1 + magnitude), cmap='viridis')
axes[0, 1].set_title("Original Magnitude")
axes[0, 1].axis('off')

# Image phase
axes[0, 2].imshow(phase, cmap='viridis')
axes[0, 2].set_title("Original Phase")
axes[0, 2].axis('off')

# Scaled image
axes[1, 0].imshow(image_scaled, cmap='gray')
axes[1, 0].set_title(f"Scaled Image (factor {scale_factor})")

# Scaled image magnitude
axes[1, 1].imshow(np.log(1 + magnitude_scaled), cmap='viridis')
axes[1, 1].set_title("Scaled Magnitude")
axes[1, 1].axis('off')

# Scaled image phase
axes[1, 2].imshow(phase_scaled, cmap='viridis')
axes[1, 2].set_title("Scaled Phase")
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Compute 2D Fourier Transform
dft = np.fft.fft2(image_bw)
dft_shifted = np.fft.fftshift(dft)
amplitude = np.abs(dft_shifted)
phase = np.angle(dft_shifted)

# Create filters for horizontal and vertical variations
rows, cols = image_bw.shape
crow, ccol = rows // 2, cols // 2

# Filter for vertical variations (keeps central horizontal frequencies)
vert_mask = np.zeros((rows, cols), np.uint8)
vert_mask[:, ccol-5:ccol+5] = 1

# Filter for horizontal variations (keeps central vertical frequencies)
horz_mask = np.zeros((rows, cols), np.uint8)
horz_mask[crow-5:crow+5, :] = 1

# Apply filters
dft_vert = dft_shifted * vert_mask
dft_horz = dft_shifted * horz_mask

# Reconstruct filtered images
def reconstruct_image(complex_dft):
    complex_dft_unshifted = np.fft.ifftshift(complex_dft)
    reconstructed = np.fft.ifft2(complex_dft_unshifted)
    return np.real(reconstructed)

image_vert = reconstruct_image(dft_vert)
image_horz = reconstruct_image(dft_horz)

# Normalize for visualization
def normalize_image(img):
    return (img - np.min(img)) / (np.max(img) - np.min(img))

# Extract and normalize variations
vertical_variations = np.diff(image_bw, axis=0)
vertical_variations = np.vstack([vertical_variations, np.zeros((1, image_bw.shape[1]))])
vertical_variations_norm = normalize_image(vertical_variations)

horizontal_variations = np.diff(image_bw, axis=1)
horizontal_variations = np.hstack([horizontal_variations, np.zeros((image_bw.shape[0], 1))])
horizontal_variations_norm = normalize_image(horizontal_variations)

# Prepare normalized images
image_norm = normalize_image(image_bw)
image_vert_norm = normalize_image(image_vert)
image_horz_norm = normalize_image(image_horz)

# Interactive visualization
def plot_fft_variations(variation_type):
    plt.figure(figsize=(24, 8))  # Increased figure size

    if variation_type == "Vertical Variations":
        images = [image_norm, vertical_variations_norm, image_vert_norm, np.log(1 + np.abs(dft_vert)), np.angle(dft_vert)]
        titles = ["Original Image", "Modified image (only Vertical Variations)", "Image without Vertical Variations", "Modified Image Magnitude", "Modified Image Phase"]
    else:
        images = [image_norm, horizontal_variations_norm, image_horz_norm, np.log(1 + np.abs(dft_horz)), np.angle(dft_horz)]
        titles = ["Original Image", "Modified image (only Horizontal Variations)", "Image without Horizontal Variations", "Modified Image Magnitude", "Modified Image Phase"]

    for i in range(5):
        plt.subplot(1, 5, i + 1)
        plt.imshow(images[i], cmap='gray' if i < 3 else 'viridis')
        plt.title(titles[i])
        plt.axis('off')

    plt.tight_layout()
    plt.show()

# Dropdown menu
dropdown = Dropdown(
    options=["Vertical Variations", "Horizontal Variations"],
    value="Vertical Variations",
    description="Select:",
    layout=widgets.Layout(width='30%')
)

interact(plot_fft_variations, variation_type=dropdown);

This tool provides an intuitive way to grasp how manipulating the magnitude spectrum impacts the reconstructed image.

🔗 **[Live 2D FFT Visualization](https://monman53.github.io/2dfft/)**  

## Phase is the key

While it might seem intuitive to think that the **magnitude spectrum** carries most of the information about an image, experiments show that **the phase is actually the key component for reconstruction**. The phase spetrum encodes most of the structural and spatial information of an image.

Creating **hybrid images** by swapping or combining magnitude and phase between different images:

1. **The reconstructed image looks much more like the image from which the phase was taken, rather than the magnitude.**  
   - This demonstrates that **the phase spectrum contains the primary structure of an image**, while the magnitude spectrum influences contrast and intensity distribution.

2. **Magnitude alone does not carry the detailed image information.**  
   - If an image is reconstructed using only the magnitude of one image and the phase of another, the result looks **more like the original phase image**.

Thus, the phase spectrum is crucial for preserving the spatial relationships and structure of an image, whereas the magnitude primarily influences intensity variations.


In [ ]:
!wget -O url1.jpg "https://upload.wikimedia.org/wikipedia/commons/1/15/Red_Apple.jpg"
!wget -O url2.jpg "https://upload.wikimedia.org/wikipedia/commons/3/36/Kyoho-grape.jpg"

In [ ]:
def reconstruct_image(magnitude, phase):
    complex_signal = magnitude * np.exp(1j * phase)
    complex_signal = np.fft.ifftshift(complex_signal)
    reconstructed = np.fft.ifft2(complex_signal)
    return np.real(reconstructed)

def normalize_image(img):
    return (img - np.min(img)) / (np.max(img) - np.min(img)) * 255

# Load and convert in greyscale the images
image1 = cv2.cvtColor(cv2.imread("url1.jpg"), cv2.COLOR_BGR2GRAY)
image2 = cv2.cvtColor(cv2.imread("url2.jpg"), cv2.COLOR_BGR2GRAY)

# Resize images to the same dimensions
min_shape = (min(image1.shape[0], image2.shape[0]), min(image1.shape[1], image2.shape[1]))
image1 = cv2.resize(image1, (min_shape[1], min_shape[0]), interpolation=cv2.INTER_AREA)
image2 = cv2.resize(image2, (min_shape[1], min_shape[0]), interpolation=cv2.INTER_AREA)

# Apply DFT
dft1 = np.fft.fft2(image1)
dft2 = np.fft.fft2(image2)
dft1_shifted = np.fft.fftshift(dft1)
dft2_shifted = np.fft.fftshift(dft2)

magnitude1 = np.abs(dft1_shifted)
magnitude2 = np.abs(dft2_shifted)
phase1 = np.angle(dft1_shifted)
phase2 = np.angle(dft2_shifted)

# Create hybrids

# Magnitude from Image 1 + Phase from Image 2
hybrid1 = reconstruct_image(magnitude1, phase2)

# Magnitude from Image 2 + Phase from Image 10
hybrid2 = reconstruct_image(magnitude2, phase1)

# Sum of magnitudes + Phase from Image 1
magnitude_sum = (magnitude1 + magnitude2) / 2
hybrid3 = reconstruct_image(magnitude_sum, phase1)

# Magnitude from Image 1 + Sum of phases
phase_sum = (phase1 + phase2) / 2
hybrid4 = reconstruct_image(magnitude1, phase_sum)

hybrid1 = normalize_image(hybrid1)
hybrid2 = normalize_image(hybrid2)
hybrid3 = normalize_image(hybrid3)
hybrid4 = normalize_image(hybrid4)

# Figure
plt.figure(figsize=(16, 12))

# First row: original images
plt.subplot(3, 2, 1)
plt.imshow(image1, cmap='gray')
plt.title('Image 1 (Apple)')
plt.axis('off')

plt.subplot(3, 2, 2)
plt.imshow(image2, cmap='gray')
plt.title('Image 2 (Grape)')
plt.axis('off')

# Second row: Hybrids
plt.subplot(3, 2, 3)
plt.imshow(hybrid1, cmap='gray')
plt.title('Magnitude Apple + Phase Grape')
plt.axis('off')

plt.subplot(3, 2, 4)
plt.imshow(hybrid2, cmap='gray')
plt.title('Magnitude Grape + Phase Apple')
plt.axis('off')

# Third row: new hybrids with sums
plt.subplot(3, 2, 5)
plt.imshow(hybrid3, cmap='gray')
plt.title('Sum of magnitudes + Phase Apple')
plt.axis('off')

plt.subplot(3, 2, 6)
plt.imshow(hybrid4, cmap='gray')
plt.title('Magnitude Apple + Sum of Phases')
plt.axis('off')

plt.tight_layout()
plt.show()

## Why Convolve When You Can Multiply?

One of the biggest advantages of using the **DFT in image processing** is how it simplifies **convolution**. Instead of performing **complex spatial operations**, we can simply **multiply** in the frequency domain.  

$$G = \mathcal{F}\{g\} = \mathcal{F}\{h \star i\} = \mathcal{F}\{h\}\mathcal{F}\{i\}= H * I$$

The difference between the two scenario is marked:
- Convolution in the **Spatial Domain**  
  - Defined as the sum of **weighted neighboring pixels** using a filter kernel
  - Requires **sliding a kernel** over the image, performing multiple multiplications and summations.Computationally **expensive** for large kernels and images.

- Convolution in the **Frequency Domain**  
  - Compute the **DFT** of the image $I$.  
  - Compute the **DFT** of the filter kernel $H$.  
  - Multiply their frequency representations:  
  - Apply the **Inverse DFT** to get the final filtered image.  

This drastically reduces computational complexity, making it **ideal for large filters**.

<div align="center">
    <img src="https://drive.google.com/uc?export=download&id=1fpFr7TfERlwOMBmg8McZIT8YIyVtpPMF" width="500">
</div>

In [ ]:
from scipy import signal

fig = None
axes = None
image = cv2.imread('image.jpg')
image_bw = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

def plot_equivalence(img, kernel_type='gaussian'):
    global fig, axes

    # Create different kernels
    if kernel_type == 'gaussian':
        kernel = cv2.getGaussianKernel(25, 5)
        kernel = kernel @ kernel.T
        title = "Gaussian Blur"
    elif kernel_type == 'high-pass':
        kernel = np.ones((25, 25)) / (25*25)
        kernel = -kernel
        kernel[12, 12] = 1 + kernel[12, 12]
        title = "High-Pass Filter"
    elif kernel_type == 'low-pass':
        kernel = np.ones((25, 25)) / (25*25)
        title = "Low-Pass Filter"
    elif kernel_type == 'band-pass':
        kernel_low = cv2.getGaussianKernel(25, 5)
        kernel_low = kernel_low @ kernel_low.T
        kernel_high = cv2.getGaussianKernel(25, 1)
        kernel_high = kernel_high @ kernel_high.T
        kernel = kernel_low - kernel_high
        kernel = kernel / np.sum(np.abs(kernel))
        title = "Band-Pass Filter"
    elif kernel_type == 'edge-detect':
        kernel = np.array([[-1, -2, -1],
                           [0,  0,  0],
                           [1,  2,  1]])
        title = "Edge Detection"


    original_kernel = kernel.copy()
    kernel = kernel / np.max(np.abs(kernel))
    padded_kernel = np.zeros_like(img, dtype=float)
    kh, kw = kernel.shape
    padded_kernel[:kh, :kw] = kernel

    # Spatial convolution
    img_float = img.astype(float)
    if img_float.max() > 1.0:  # Normalize if needed
        img_float = img_float / 255.0

    spatial_result = signal.convolve2d(img_float, kernel, mode='same', boundary='wrap')
    if kernel_type == 'edge-detect':
        spatial_result = np.abs(spatial_result)
        spatial_result = spatial_result / np.max(spatial_result)
        spatial_result = np.power(spatial_result, 0.5)  # Apply gamma correction to enhance edges

    # Frequency domain
    img_fft = np.fft.fft2(img_float)
    img_fft_shifted = np.fft.fftshift(img_fft)

    # Kernel DFT
    kernel_fft = np.fft.fft2(np.fft.ifftshift(padded_kernel))
    kernel_fft_shifted = np.fft.fftshift(kernel_fft)

    # Convolution in frequency domain is multiplication
    freq_result_fft = img_fft * kernel_fft
    freq_result = np.real(np.fft.ifft2(freq_result_fft))
    if kernel_type == 'edge-detect':
        freq_result = np.abs(freq_result)
        freq_result = freq_result / np.max(freq_result)
        freq_result = np.power(freq_result, 0.5)  # Apply gamma correction to enhance edges

    img_magnitude = np.log(1 + np.abs(img_fft_shifted))
    kernel_magnitude = np.log(1 + np.abs(kernel_fft_shifted))
    result_magnitude = np.log(1 + np.abs(np.fft.fftshift(freq_result_fft)))

    img_magnitude = img_magnitude / np.max(img_magnitude)
    kernel_magnitude = kernel_magnitude / np.max(kernel_magnitude)
    result_magnitude = result_magnitude / np.max(result_magnitude)

    # Create a new figure or clean the existing one
    if fig is None or not plt.fignum_exists(fig.number):
        fig, axes = plt.subplots(2, 3, figsize=(14, 9))
        plt.subplots_adjust(wspace=0.4, hspace=0.3, left=0.05, right=0.95, top=0.95, bottom=0.05)
    else:
        for ax_row in axes:
            for ax in ax_row:
                ax.clear()

    axes[0, 0].imshow(img_float, cmap='gray')
    axes[0, 0].set_title('Original Image')
    axes[0, 0].axis('off')

    # Visualization stuff
    if kernel_type == 'low-pass':
        display_kernel = original_kernel / np.max(original_kernel)
    else:
        display_kernel = original_kernel

    axes[0, 1].imshow(display_kernel, cmap='gray')
    axes[0, 1].set_title(f'Kernel: {title}')
    axes[0, 1].axis('off')

    axes[0, 2].imshow(spatial_result, cmap='gray')
    axes[0, 2].set_title('Spatial Convolution Result')
    axes[0, 2].axis('off')

    axes[1, 0].imshow(img_magnitude, cmap='viridis')
    axes[1, 0].set_title('Image DFT Magnitude')
    axes[1, 0].axis('off')

    axes[1, 1].imshow(kernel_magnitude, cmap='viridis')
    axes[1, 1].set_title('Kernel DFT Magnitude')
    axes[1, 1].axis('off')

    axes[1, 2].imshow(result_magnitude, cmap='viridis')
    axes[1, 2].set_title('Frequency Domain Result Magnitude')
    axes[1, 2].axis('off')

    if hasattr(fig, 'patches'):
        for patch in fig.patches[:]:
            if isinstance(patch, FancyArrowPatch):
                patch.remove()

    for artist in fig.texts[:]:
        artist.remove()

    fig.text(0.325, 0.755, '$\\star$', fontsize=24)  # Spatial convolution symbol
    fig.text(0.330, 0.235, '$\\cdot$', fontsize=24)  # Frequency multiplication symbol
    fig.text(0.655, 0.755, '$=$', fontsize=24)       # Spatial result equals
    fig.text(0.653, 0.235, '$=$', fontsize=24)       # Frequency result equals

    arrow1 = FancyArrowPatch(
        (0.22, 0.56), (0.22, 0.44),
        connectionstyle="arc3,rad=0",
        arrowstyle="-|>",
        mutation_scale=15,
        color='blue',
        transform=fig.transFigure
    )
    fig.text(0.18, 0.5, '$\\mathcal{F}$', fontsize=18)

    arrow2 = FancyArrowPatch(
        (0.78, 0.44), (0.78, 0.56),
        connectionstyle="arc3,rad=0",
        arrowstyle="-|>",
        mutation_scale=15,
        color='red',
        transform=fig.transFigure
    )
    fig.text(0.80, 0.5, '$\\mathcal{F}^{-1}$', fontsize=18)

    fig.patches.append(arrow1)
    fig.patches.append(arrow2)

    # Update the figure
    plt.draw()

    return spatial_result, freq_result

def interactive_convolution(image):
    global fig, axes

    # Create dropdown widget
    kernel_dropdown = widgets.Dropdown(
        options=['gaussian', 'high-pass', 'low-pass', 'band-pass', 'edge-detect'],
        value='gaussian',
        description='Kernel Type:',
        style={'description_width': 'initial'}
    )

    def update_plot(change):
        clear_output(wait=True)

        display(kernel_dropdown)
        plot_equivalence(image, change['new'])

    kernel_dropdown.observe(update_plot, names='value')
    display(kernel_dropdown)
    plot_equivalence(image, 'gaussian')

interactive_convolution(image_bw)